The __textblob.classifiers__ module makes it simple to create custom classifiers.
As an example, Let's create a custom sentiment analyzer.

### Loading Data and Creating a Classifier
First we'll create some training and test data.

In [1]:
train = [('I love this sandwich.', 'pos'),
 ('this is an amazing place!', 'pos'),
 ('I feel very good about these beers.', 'pos'),
 ('this is my best work.', 'pos'),
 ('what an awesome view', 'pos'),
 ('I do not like this restaurant', 'neg'),
 ('I am tired of this stuff.', 'neg'),
 ("I can't deal with this", 'neg'),
 ('he is my sworn enemy!', 'neg'),
 ('my boss is horrible.', 'neg')]

In [5]:
test = [('the beer was good.', 'pos'),
 ('I do not enjoy my job', 'neg'),
 ("I ain't feeling dandy today.", 'neg'),
 ('I feel amazing!', 'pos'),
 ('Gary is a friend of mine.', 'pos'),
 ("I can't believe I'm doing this.", 'neg')]

Create a __Naive Bayes classifier__, passing the training data into the constructor.

In [6]:
from textblob.classifiers import NaiveBayesClassifier

In [8]:
cl = NaiveBayesClassifier(train)

<NaiveBayesClassifier trained on 10 instances>

### Loading data from files
You can also load data from common file formats including CSV, JSON, and TSV.

CSV files should be formatted like so:
```csv
I love this sandwich.,pos
This is an amazing place!,pos
I do not like this restaurant,neg
```